In [37]:
import pandas as pd
import numpy as np
import requests
import json
from pandas.io.json import json_normalize

#### Retorna a categoria do local

In [41]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    return categories_list[0]['name']

#### Informações para acessar a API do foursquare

In [70]:
CLIENT_ID = '55LWIFZLZNZPSTAZZQKJXBD11BSBCUZTZPIXWVV4RFZJP1HS' # your Foursquare ID
CLIENT_SECRET = 'RPCOWGHQY5YNQUCHQUTC5VLKEYFGZN15FSIYTK5GNXNZ301E' # your Foursquare Secret
VERSION = '20180604'

#### Informações da cidade delimitada

In [144]:
latitude_nyork = 40.730610
longitude_nyork = -73.935242
radius = 50000
LIMIT = 2500

#### Coletando as informações dos locais

In [145]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(
    CLIENT_ID, CLIENT_SECRET, latitude_nyork, longitude_nyork, VERSION, radius, LIMIT)
            
# make the GET request
results = requests.get(url).json()

In [146]:
venues = results['response']['groups'][0]['items']  
nearby_venues = json_normalize(venues)
filtered_columns = ['venue.id', 'venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.tail(10)

,id,name,categories,lat,lng
90,4a66f31af964a520dcc81fe3,Equinox West 50th Street,Gym,40.761912,-73.984268
91,51971065498ef767dfaa6b6c,Dynaco,Bar,40.686660,-73.954633
92,546232fb498e6f99835ec2c2,Stick With Me,Chocolate Shop,40.721295,-73.995627
93,557dae53498ed2645c3ebbc0,Mekelburg's,Gourmet Shop,40.687571,-73.962370
94,52701b99498eaa2785f4f68f,Winter Garden Theatre,Theater,40.761405,-73.983555
95,59e0ee84f0ca95526b7fce9f,CAVA,Mediterranean Restaurant,40.721928,-73.996512
96,4a06f82df964a52010731fe3,Two Little Red Hens,Bakery,40.777523,-73.951761
97,4b1074dbf964a520d47023e3,Majestic Theatre,Theater,40.758298,-73.988154
98,4ed14ab093ad987b4e200de1,Speedy Romeo,Pizza Place,40.687397,-73.959879
99,3fd66200f964a520b6e71ee3,Carnegie Hall,Concert Hall,40.765125,-73.979918


#### Para cada local, encontrar as pessoas que curtiram

In [147]:
users_total = pd.DataFrame()
for venue in nearby_venues.iterrows():
    url = 'https://api.foursquare.com/v2/venues/{}/likes?client_id={}&client_secret={}&v={}&limit={}'.format(
    venue[1]['id'], CLIENT_ID, CLIENT_SECRET, VERSION, 1000)
    results = requests.get(url).json()
    results_2 = results['response']['likes']['items']
    users = json_normalize(results_2)[['id', 'firstName', 'gender']]
    users['avenue_id'] = venue[1]['id']
    users['avenue_name'] = venue[1]['name']
    users['avenue_categories'] = venue[1]['categories']
    users['avenue_lat'] = venue[1]['lat']
    users['avenue_lng'] = venue[1]['lng']
    if users_total.empty:
        users_total = users
    else:
        users_total = pd.concat([users_total, users], ignore_index=True)

In [148]:
users_total.tail()

,id,firstName,gender,avenue_id,avenue_name,avenue_categories,avenue_lat,avenue_lng
55686,12008009,Mert,male,3fd66200f964a520b6e71ee3,Carnegie Hall,Concert Hall,40.765125,-73.979918
55687,46043188,Christopher,male,3fd66200f964a520b6e71ee3,Carnegie Hall,Concert Hall,40.765125,-73.979918
55688,66093767,Sun-Young,female,3fd66200f964a520b6e71ee3,Carnegie Hall,Concert Hall,40.765125,-73.979918
55689,1876689,Virginia,female,3fd66200f964a520b6e71ee3,Carnegie Hall,Concert Hall,40.765125,-73.979918
55690,53586928,Debora,female,3fd66200f964a520b6e71ee3,Carnegie Hall,Concert Hall,40.765125,-73.979918


In [149]:
len(users_total)

55691

In [150]:
users_total.to_csv('curtidas_new_york.csv', index=False)